In [2]:
!pip install pyspark
!pip install neo4j

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Ejemplo de PySpark en Jupyter Notebook") \
    .getOrCreate()

# Obtener el SparkContext
sc = spark.sparkContext

In [4]:
graph = [(1,11,2),(1,11,3),(2,11,3),(3,11,2),(3,11,4),(4,11,1),(4,11,2),(4,11,3),(4,12,5),(5,12,1),(5,12,2),(5,12,6)]
#graph = [(2,11,2), (1,11,2), (2,11,1)]

In [5]:
B = 2

In [6]:
def hash(x):
    return x % B

In [7]:
def map_pdm(x, arista, y):

    x_hash = hash(x)
    y_hash = hash(y)

    l = []
    for i in range(B):
        l.append(((x_hash, y_hash, i), (x, arista, y)))
        l.append(((i, x_hash, y_hash), (x, arista, y)))
        l.append(((y_hash, i, x_hash), (x, arista, y)))

    return set(l)

In [8]:
datos_rdd = sc.parallelize(graph)

### Pasarlo por map

In [9]:
resultado_rdd = datos_rdd.flatMap(lambda dato: map_pdm(*dato))

resultado_rdd.collect()[0:2]

[((1, 0, 0), (1, 11, 2)), ((0, 1, 1), (1, 11, 2))]

### Reduce


In [20]:
test = resultado_rdd.groupByKey().mapValues(list)
test.collect()[0]

((1, 0, 0),
 [(1, 11, 2),
  (2, 11, 3),
  (3, 11, 2),
  (3, 11, 4),
  (4, 11, 1),
  (4, 11, 2),
  (4, 11, 3),
  (4, 12, 5),
  (5, 12, 2),
  (5, 12, 6)])

In [28]:
def triangulo(nodos):

  final = []
  for i in range(len(nodos)):
    trio_2 = nodos[i]

    for j in range(i, len(nodos)):
      if j + 1 >= len(nodos):
        break
      sig = nodos[j]
      sub_sig = nodos[j + 1]


      if trio_2[2] == sig[0] and sig[2] == sub_sig[0] and trio_2[0] == sub_sig[2]:
        final.append((trio_2[0], sig[0], sub_sig[0]))

      elif trio_2[2] == sub_sig[0] and sig[0] == sub_sig[2] and trio_2[0] == sig[2]:
        final.append((trio_2[0], sig[0], sub_sig[0]))

  return final

In [44]:
final = test.map(lambda nodos: (nodos[0], triangulo(nodos[1])))
final.collect()

[((1, 0, 0), []),
 ((0, 1, 0), []),
 ((0, 0, 1), []),
 ((1, 1, 1), []),
 ((0, 1, 1), [(1, 3, 4)]),
 ((1, 0, 1), [(1, 3, 4)]),
 ((1, 1, 0), [(1, 3, 4)]),
 ((0, 0, 0), [])]

In [41]:
dicc = {}
for i in final.collect():
    dicc[i[0]] = i[1]

In [46]:
dicc

{(1, 0, 0): [],
 (0, 1, 0): [],
 (0, 0, 1): [],
 (1, 1, 1): [],
 (0, 1, 1): [(1, 3, 4)],
 (1, 0, 1): [(1, 3, 4)],
 (1, 1, 0): [(1, 3, 4)],
 (0, 0, 0): []}